In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 MB 294.8 kB/s eta 0:00:0000:0100:22
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 292.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 438.7 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 416.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 459.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 312.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 624.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 440.8 kB/s eta 0:00:0000:0100:01


In [4]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 593.0 kB/s eta 0:00:00 0:00:01


In [7]:
import csv
import re
#import emoji
import torch
from tensorflow import keras
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertForSequenceClassification, AdamW, BertConfig
from tqdm import tqdm, trange
import numpy as np
import time
import random
import pandas as pd 
import datetime
from _datetime import datetime as dt
from sklearn.metrics import classification_report
from transformers import AutoModel, AutoTokenizer, AutoModelWithLMHead
from sklearn.model_selection import train_test_split
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForMaskedLM,
    BertTokenizer,
    #CamembertConfig,
    #CamembertForMaskedLM,
    #CamembertTokenizer,
    DistilBertConfig,
    DistilBertForMaskedLM,
    DistilBertTokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTConfig,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    #get_linear_schedule_with_warmup,
)

In [8]:


def load_train_test_data1(tokenizer,Text):

    Data=[]
    count=0
    Label=[]
    for i in Text:
        #print (i)
        full_line = str(i)
        #full_line = re.sub(r'#([^ ]*)', r'\1', full_line)
        full_line = re.sub(r'https.*[^ ]', 'URL', full_line)
        full_line = re.sub(r'http.*[^ ]', 'URL', full_line)
        #full_line = emoji.demojize(full_line)
        full_line = re.sub(r'(:.*?:full_line)', r' \1 ', full_line)
        full_line = re.sub(' +', ' ', full_line)
        full_line=re.sub("\s\s+" , " ", full_line)
        full_line = re.sub( '[^a-zA-Z]', ' ', full_line)
        if len(list(full_line.split()))>1:
            #print(f'{count}.{full_line}')
            Data.append(full_line)
#Label=list(df5['Labels'])

    
    test_tweets=Data
    # List of all tokenized tweets
    test_input_ids = []

    # For every tweet in the test set
    for sent in test_tweets:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
            sent,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=100, # orignal value 512 - change Tommaso

            # This function also supports truncation and conversion
            # to pytorch tensors, but we need to do padding, so we
            # can't use these features :( .
            # max_length = 128,          # Truncate all sentences.
            # return_tensors = 'pt',     # Return pytorch tensors.
        )

        # Add the encoded tweet to the list.
        test_input_ids.append(encoded_sent)

    # # Pad our input tokens with value 0.
    # # "post" indicates that we want to pad and truncate at the end of the sequence,
    # # as opposed to the beginning.

    test_input_ids = pad_sequences(test_input_ids, maxlen=100, dtype="long",
                                    value=tokenizer.pad_token_id, truncating="pre", padding="pre")



    # Create attention masks
    # The attention mask simply makes it explicit which tokens are actual words versus which are padding
    test_attention_masks = []

    # For each tweet in the test set
    for sent in test_input_ids:
        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]

        # Store the attention mask for this sentence.
        test_attention_masks.append(att_mask)

    # Return the list of encoded tweets, the list of labels and the list of attention masks
    return  test_input_ids, test_attention_masks

In [5]:
x_test[:10]

['                   you ve been a good as friend to me   glad I got your honkie ass  Let s fuck shit up this year',
 'Scandal  A Negro Bed Wench s Fairy tale and propaganda  URL',
 'when bitches change and start acting like the nigga they with  lt  lt  lt  lt  lt  lt ',
 'She even had a hoe name ',
 'Golden Oreos are the best',
 ' VNNForum These Marxist kikes promoted Anti Semitism by creating this film  what was their incentive  They just promote their enemies ',
 ' USER  USER  USER  USER He is an awesome dog  Almost worth the trip',
 ' USER Yes everyone does but he is in the best position and missing all over the place',
 ' GrizzboAdams ight nigga haha fucking queer has to clear his schedule',
 ' USER  USER I agree he is very intelligent and I like the latest policy of a sovereign wealth fund   something I ve long supported  But he just doesn t cut through  Someone like Jo Swinson would get the party more air time imo ']

In [10]:

#model_dir='../Data/2023-01-04_Covidtweets'
model_dir='2022-12-29_Hatedata'

Text=["when bitches change and start acting like the nigga they with  lt  lt  lt  lt  lt  lt ", 'Scandal  A Negro Bed Wench s Fairy tale and propaganda  URL']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained Tokenizer from directory, change this to load a tokenizer from ber package
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(model_dir)
#tokenizer = tokenizer_class.from_pretrained(model_dir)

# Copy the model to the GPU.
#model.to(device)

# Set the model to work on CPU if no GPU is present
model.to(device)

validation_inputs, validation_masks = load_train_test_data1(tokenizer,Text)


# Tweets
validation_inputs = torch.tensor(validation_inputs)

# Attention masks
validation_masks = torch.tensor(validation_masks)


batch_size = 1

# Create the DataLoader for our validation set.
#validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_data = TensorDataset(validation_inputs, validation_masks)
validation_sampler = SequentialSampler(validation_data)
# Note that the number of batch has to be the same, this means that we have to aggregate results in the end
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)



# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

    # Store  predicted labels for global eval
predicted_labels = []

t0 = time.time()
model.eval()

    # Tracking variables
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
for batch in validation_dataloader:
        # Add batch to GPU/CPU
    batch = tuple(t.to(device) for t in batch)
    #batch=tuple(t.to(device))

        # Unpack the inputs from our dataloader
    #b_input_ids, b_input_mask, b_labels = batch
    b_input_ids, b_input_mask = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
    with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
    logits = outputs[0]

        # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    #label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
    #tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
    #eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
    #nb_eval_steps += 1

    pred_flat = np.argmax(logits, axis=1).flatten()
    #labels_flat = label_ids.flatten()

       # Store gold labels single list
    #gold_labels.extend(labels_flat)
        # Store predicted labels single list
    predicted_labels.extend(pred_flat)
count=0
for i,j in zip(predicted_labels,Text):
    count=count+1
    if (i==0):
        print (f"{count}. {j}= Non-Hate")
    else:
        print (f"{count}. {j}= Hate")
#print (predicted_labels[0])


1. when bitches change and start acting like the nigga they with  lt  lt  lt  lt  lt  lt = Hate
2. Scandal  A Negro Bed Wench s Fairy tale and propaganda  URL= Hate
